In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread("IMG-2012.jpg")
img = cv2.resize(img, (int(480*2), int(640*2)))

In [3]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gri Tonlama", gray_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 1)
cv2.imshow("Blurlama", blurred_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
canny_img = cv2.Canny(blurred_img, 190, 190)
cv2.imshow("Kenar Algilama", canny_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
contours, _ = cv2.findContours(canny_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [7]:
max_area = 0
best_corners = None
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 500:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4 and area > max_area:
            max_area = area
            best_corners = approx


In [8]:
contour_frame = img.copy()
cv2.drawContours(contour_frame, [best_corners], -1, (0, 255, 0), 2)

array([[[150, 176, 183],
        [155, 181, 187],
        [158, 184, 189],
        ...,
        [181, 201, 206],
        [174, 194, 200],
        [181, 201, 206]],

       [[155, 181, 187],
        [156, 181, 188],
        [156, 182, 188],
        ...,
        [182, 202, 208],
        [171, 191, 196],
        [179, 199, 204]],

       [[156, 182, 188],
        [153, 179, 185],
        [151, 177, 183],
        ...,
        [181, 201, 206],
        [185, 205, 210],
        [182, 202, 207]],

       ...,

       [[ 33,  43,  43],
        [ 32,  42,  42],
        [ 26,  36,  36],
        ...,
        [185, 204, 209],
        [186, 205, 210],
        [185, 204, 209]],

       [[ 38,  48,  48],
        [ 33,  43,  43],
        [ 30,  40,  40],
        ...,
        [186, 205, 210],
        [184, 203, 208],
        [186, 205, 210]],

       [[ 36,  46,  46],
        [ 42,  52,  52],
        [ 57,  67,  67],
        ...,
        [184, 203, 208],
        [184, 203, 208],
        [180, 199, 204]]

In [9]:
if best_corners is not None:
    # Köşeleri düzeltme için sıralama
    best_corners = np.squeeze(best_corners)
    ordered_corners = np.zeros((4, 2), dtype=np.float32)
    sum_of_coordinates = np.sum(best_corners, axis=1)
    ordered_corners[0] = best_corners[np.argmin(sum_of_coordinates)]
    ordered_corners[2] = best_corners[np.argmax(sum_of_coordinates)]
    diff_of_coordinates = np.diff(best_corners, axis=1)
    ordered_corners[1] = best_corners[np.argmin(diff_of_coordinates)]
    ordered_corners[3] = best_corners[np.argmax(diff_of_coordinates)]

    # Düzleştirme işlemi
    width, height = 480, 640
    target_corners = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])
    transform_matrix = cv2.getPerspectiveTransform(ordered_corners, target_corners)
    scanned_img = cv2.warpPerspective(img, transform_matrix, (width, height))

    # Görüntüleri gösterme
    cv2.imshow("Orijinal Resim", img)
    cv2.imshow("Taratilan Resim", scanned_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Belge bulunamadı.")